# Single-Node Rendering of a Numpy Array in ParaView
There are a ton of great visualization frameworks out there for Jupyter and iPython. Most of these work by importing data from the python kernel on the server back to the client to be visualized with Javascript. This has proven to be a very capable and flexible approach, but it's not without its drawbacks. As your data size grows, the cost to transfer between server and client becomes prohibitive--and that's if your client machine can even handle the data size to begin with. Usually, data in this category is aggregated or decimated in some way before visualization. But, if you happen to be working on a server with a big, beefy GPU, wouldn't it be nice to be able to just render the data where it lives?

Server-side remote rendering, both single- and multi-node, has been standard practice in the scientific simulation community for years, supported by a set of feature-rich tools. This project brings one of the most popular scientific visualization packages, ParaView, into the Jupyter notebook.

We'll work with a CT scan of a human head for our example. We'll first download it from a public source, then load it into memory using numpy. Since the data is in a raw binary format, we have to know the format ahead of time--16-bit unsigned integer, big-endian, and 256x256x113 grid dimensions. Once we have the data in a numpy array, we can use VTK's built-in numpy support to wrap the array in a vtkImageData object.

In [ ]:
# Make sure we're able to import urllib.request
try:
    from urllib.request import urlretrieve
except ImportError:
    from urllib import urlretrieve
import os

# Check for a cached copy of the dataset, and download it if needed
filename = 'CT-Head.raw'
if not os.path.exists(filename):
    url = 'https://github.com/aashish24/vtkvolume-data/raw/master/CT-Head/CT-Head.raw'
    urlretrieve(url, filename)

# General pre-reqs
import numpy as np

# Load raw volume from disk using numpy
npvol = np.array(np.fromfile(filename, dtype=np.uint16)).byteswap(inplace=True)
voldims = [256,256,113]

# Pre-reqs for numpy-->vtk and paraview visualization
import vtk
from vtk.util import numpy_support as vtknp

# Create a VTK image to hold our data -- NOTE: ParaView is *very* particular about how this is set up
vtkimg = vtk.vtkImageData()
vtkimg.SetDimensions(voldims)
vtkimg.SetExtent([0,voldims[0]-1, 0,voldims[1]-1, 0,voldims[2]-1])
vtkimg.SetSpacing([1.0/voldims[0],1.0/voldims[1],1.0/voldims[2]])
vtkimg.SetOrigin([-0.5,-0.5,-0.5])

# Get a VTK wrapper around the numpy volume
dataName = 'CTHead'
vtkarr = vtknp.numpy_to_vtk(npvol)
vtkarr.SetName(dataName)
vtkimg.GetPointData().AddArray(vtkarr)
vtkimg.GetPointData().SetScalars(vtkarr)

With our VTk object in hand, we can initialize the ParaView state. We create three state objects--a Producer, a RenderView, and a Filter--as part of our ParaView pipeline. Since we already have our data in a VTK format, then we can just wrap it in a TrivialProducer to get it into the ParaView pipeline. The RenderView object serves as our window onto the data, and is also the way we change the resolution, camera position, etc. Finally, we add a "Contour" filter to ParaView's pipeline. This takes our input volume and computes an iso-value surface as a 3D mesh.

In [ ]:
import paraview
from paraview.simple import *

# Wrap our vtkImageData in a proxy object, then add it to the pipeline
trivprod = TrivialProducer()
trivprod.GetClientSideObject().SetOutput(vtkimg)
trivprod.UpdatePipeline()

paraview.simple._DisableFirstRenderCameraReset() #keeps ParaView from changing the camera settings on us

# Create the primary 'Render View'
renV = CreateView('RenderView')
renV.ViewSize = [800, 500] # Resolution
renV.CameraPosition = [0, 0, 2.5]   # Initial camera position
renV.Background = [0, 0, 0] # Set the background to black

# create a new 'Contour' filter to compute an isosurface from our volume
contour = Contour(Input=trivprod)
contour.ContourBy = ['POINTS', dataName]
contour.Isosurfaces = [1627]
contour.UpdatePipeline()

# transform the mesh for easier viewer
transform = Transform(Input=contour)
transform.Transform.Translate = [0.1,0,0]
transform.Transform.Rotate = [-90.0, -180.0, -180.0]
transform.UpdatePipeline()

# finally, create a Display object for the transformed contour and configure it's properties
contourDisplay = Show(transform, renV)
contourDisplay.Representation = 'Surface'

#set the color of the isosurface
ColorBy(contourDisplay, None)
contourDisplay.AmbientColor = contourDisplay.DiffuseColor = [1, 1, 1]

# add a few lights here, but disable them for now--we'll use them later
light1,light2,light3 = AddLight(view=renV),AddLight(view=renV),AddLight(view=renV)
light1.Enable = light2.Enable = light3.Enable = 0

Now that ParaView's render state is initialized, we can create a PVDisplay widget (passing in the RenderView object that we want to show). Click and drag with the left mouse button to rotate the view, drag with the middle mouse to pan, and use the mouse wheel to zoom.

In [ ]:
# Import the PVDisplay widget, and create it
from ipyparaview.widgets import PVDisplay
w = PVDisplay(renV)
display(w)

Let's spice it up our visuals a bit by turning on the OptiX path tracing backend. Note that since we're modifying the shared render state, this will change the visualization for all displays which use the same RenderView.

In [ ]:
renV.BackEnd = 'OptiX pathtracer'
renV.EnableRayTracing = 1
renV.Denoise = 1 #turn on denoising
renV.SamplesPerPixel = 4 #need at least 4 accumulated frames to trigger denoising

#Optional: load a materials file to allow setting materials
renV.OSPRayMaterialLibrary = GetMaterialLibrary()
renV.OSPRayMaterialLibrary.LoadMaterials= 'materials.json'

#Add a shiny ground plane to make it really obvious that we're tracing some rays
plane = Plane()
plane.Origin = [-3, -0.5, -3]
plane.Point1 = [ 3, -0.5, -3]
plane.Point2 = [-3, -0.5,  3]
planeDisplay = Show(plane, renV)
planeDisplay.Representation = 'Surface'
planeDisplay.OSPRayMaterial = 'Metal'

Since we have full access to the state, we can use iPython widgets to interactively adjust pipeline and render parameters, such as the isovalue of the contour filter. The isovalue itself is set in the callback, `iso` (which also intitiates a pipeline update), that we pass to interact. Interact infers the kind of widget we want from the provided default value and spawans a widget for us.

In [ ]:
# Interact from ipywidgets offers us a simple way to interactively control values with a callback function
from ipywidgets import interact

# sets the isosurface value, triggers a pipeline update, and renders the result
def iso(isoval):
    contour.Isosurfaces = [isoval]
    contour.UpdatePipeline()
    
interact(iso, isoval=1627.0)
# display(w)

Changing the isovalue is just scratching the surface of interative modification. A major advantage of physically-based rendering is the ability to control the lighting in order to highlight particular features of the model, or to achieve a particular aesthetic effect. This more complex example adds two directional lights and one point light, and adds some controls for direction (or position), luminance, and color.

In [ ]:
# First, we set up a few utility functions, and callbacks to control light parameters
import ipywidgets as widgets
from ipywidgets import fixed, interactive_output

# Utility function for transforming (theta,phi) to (x,y,z)
def sphereToCart(t, p):
    from math import sin, cos, pi
    t,p = pi*t/180.0, pi*p/180.0
    return [sin(p)*cos(t), sin(t), cos(p)*cos(t)]

# Utility function for transforming a hex code to an (r,g,b) triplet
def hexToRGB(h):
    return list(float(int(h.strip('#')[i:i+2], 16))/255.0 for i in (0, 2, 4))

# Callback function to set direction,color,luminance for directional lights
# dirParams = ['light','theta','phi','col','lum']
dirParams = ['light','phi','theta','col','lum']
def setDirLight(light, theta, phi, col, lum):
    light.Position = sphereToCart(theta,phi)
    light.DiffuseColor = hexToRGB(col)
    light.Intensity = lum

# Callback function to set position,color,luminance for point lights
ptParams = ['light','x','y','z','col','lum']
def setPtLight(light, x, y, z, col, lum):
    light.Position = [x,y,z]
    light.DiffuseColor = hexToRGB(col)
    light.Intensity = lum

In [ ]:
# Now we're ready to set up our lighting controls
renV.UseLight = False #turn off ParaView's default lights
light1.Enable = light2.Enable = light3.Enable = 1 #enable the three lights we added earlier
light1.Type = light2.Type = 'Directional' #lights 1 and 2 are simple directional lights
light3.Type = 'Positional' #light3 is a physical light with position, direction, cone angle, etc
light3.ConeAngle = 0.0 #turns light3 into an omni-directional point light
iso(1627) #reset the isovalue to its original

l1 = [widgets.FloatSlider(value=308.2, min=0, max=360),
      widgets.FloatSlider(value=0, min=-90, max=90),
      widgets.ColorPicker(value='#ff5700'),
      widgets.FloatLogSlider(value=1.26, base=10, min=-3, max=3, step=0.1)]

l2 = [widgets.FloatSlider(value=148.9, min=0, max=360),
      widgets.FloatSlider(value=36.7, min=-90, max=90),
      widgets.ColorPicker(value='#109ad2'),
      widgets.FloatLogSlider(value=1.0, base=10, min=-3, max=3, step=0.1)]

l3 = [widgets.FloatSlider(value=0.1, min=-1, max=1, step=0.01),
      widgets.FloatSlider(value=-0.33, min=-1, max=1, step=0.01),
      widgets.FloatSlider(value=0.5, min=-1, max=1, step=0.01),
      widgets.ColorPicker(value='#00ff00'),
      widgets.FloatLogSlider(value=0.0158, base=10, min=-3, max=1, step=0.01)]

display(widgets.HBox(l1), widgets.interactive_output(setDirLight, dict(zip(dirParams, [fixed(light1)]+l1))))
display(widgets.HBox(l2), widgets.interactive_output(setDirLight, dict(zip(dirParams, [fixed(light2)]+l2))))
display(widgets.HBox(l3), widgets.interactive_output(setPtLight, dict(zip(ptParams, [fixed(light3)]+l3))))

We can re-use the same lights to create an entirely different feel for the visualization. The ability to access the ParaView rendering state means we can easily experiment inside the notebook.

In [ ]:
# Make a metal skull \m/
contourDisplay.OSPRayMaterial = 'Metal'

# Change lights 1 & 2 to positional, light 3 to directional
light1.Type = light2.Type = 'Positional' #lights 1 and 2 become physical
light1.ConeAngle = light2.ConeAngle = 0.0 #setting an angle of 0 makes these omnidirectional
light3.Type = 'Directional'

setPtLight(light1, -0.12, 0.09, 0.24, '#ff0000', 0.891)
setPtLight(light2,  0.12, 0.09, 0.24, '#ff0000', 0.891)
setDirLight(light3, 43.6, 26.9, '#ffffff', 10.0)

<mark>TODO:</mark> Volume rendering in this example, or just in Dask-MPI example?
Of course, we're not limited to just one representation for our data. If your installation of ParaView has the NVIDIA IndeX plugin, then IndeX will automatically be used if you change the representation type to volume rendering.

In [ ]:
# renV.EnableRayTracing = 0
# ProdDisplay = Show(trivprod, renV)
# ProdDisplay.SetRepresentationType('Volume')
# Hide(contour, renV)
# Hide(plane, renV)